# データ処理プログラミング 第3回

### Livedoorニュースコーパスで学習されたモデルをもとに、ニュース記事のカテゴリを予測する

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
BASE_PATH = '/content/'

In [ ]:
!pip install transformers torch fugashi ipadic pandas tqdm

### テストデータの読み込み
- 読み込むファイル
    - `test.csv`  
    モデルのテストに利用するためのデータ。カラムと内容は以下の通りです。
    - label: ニュース記事のカテゴリを表すラベルを数字で表したもの。同じカテゴリーの記事は同じ数字になっています。
    - url: ニュース記事のURL
    - date: ニュース記事の日付
    - category: ニュース記事のカテゴリ
    - body: ニュース記事本文
- 読み込み後の形式  
    以下のような配列形式でデータを読み込んでください。
    なお、最終的にデータが格納される配列の変数名は`test_dataset`としてください。
    ```
    [
        {
            "label": ラベル, 
            "text": ニュース記事本文
        },
        {
            "label": ラベル, 
            "text": ニュース記事本文
        },
        ...
    ]
    ```


In [ ]:
# TODO: 以下にテストデータを上記の形式で読み込むコードを書いてください
import pandas as pd
df = pd.read_csv(BASE_PATH+'test.csv')
test_dataset = []
for i in range(len(df)):
    label = df.iloc[i, 0]
    text = df.iloc[i, 4]
    test_dataset.append({
        'label': label,
        'text': text
    })

### category情報が格納されたファイルの読み込み
- 読み込むファイル
    - `categories.json`  
    ラベルのカテゴリ情報が格納されたファイル。
    以下のような形式でデータが格納されています。
    ```
    [
        {
            "label": ラベル名(数字),
            "category": カテゴリ名,
        }
        ...
    ]
    
    ```
- 読み込み後の形式  
    そのままのPython dict型の要素が格納された配列で読み込んでください。
    配列が格納される変数名は`categories`としてください。

In [ ]:
# TODO: 以下にcategory情報を上記の形式で読み込むコードを書いてください
import json
with open(BASE_PATH+'categories.json') as f:
    categories = json.load(f)

### データの前処理

In [ ]:
# seed値の設定
from transformers import set_seed
seed = 42
set_seed(seed)

In [ ]:
# TODO: トークナイザを読み込むコードを書いて下さい。
# トークナイザは「'cl-tohoku/bert-base-japanese-whole-word-masking'」用のモデルを利用して下さい。この名称+tokenizer等で検索するか、学習スクリプトを参考にしてみて下さい。
# なお、読み込んだtokenizerはtokenzierという変数に格納して下さい。
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [ ]:
model_max_length = tokenizer.model_max_length

In [ ]:
# TODO: test_datasetの各辞書型のデータをfor文で回し、辞書型のデータ内の'text'キーの値をトークナイザに入力し、その結果を元の辞書型のデータに'encoding'キーで追加してください。
# tokenizerに渡すパラメーターは以下のものを利用して下さい。
# padding='max_length', truncation=True, max_length=model_max_length, return_tensors='pt'
for data in test_dataset:
    data['encoding'] = tokenizer(data['text'], padding='max_length', truncation=True, max_length=model_max_length, return_tensors='pt')

In [ ]:

# データセットの確認
print(test_dataset[0])

### モデルの読み込み

In [ ]:
# 環境の設定
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
# モデルの読み込み
from transformers import BertForSequenceClassification
import torch
model_path = BASE_PATH+'model/model.pth'
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=len(categories))
# 学習済みモデルの重みを読み込む
model.load_state_dict(torch.load(model_path))
model.eval() # モデルを評価モードにする
model.to(device) # モデルをGPUに載せる

In [ ]:
# テストデータのDataLoaderを作成
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# 評価用のスクリプト
from tqdm import tqdm
texts = []
pred_labels = []
true_labels = []
for data in tqdm(test_dataloader):
    label = data['label']
    encoding = data['encoding']
    output = model(**{k: v.squeeze(0).to(device) for k, v in encoding.items()})
    
    pred_label = torch.argmax(output.logits, dim=1)
    pred_labels.append(pred_label.item())
    true_labels.append(label)
    texts.append(data['text'])
assert len(pred_labels) == len(true_labels) 

In [ ]:
accuracy = (torch.tensor(pred_labels) == torch.tensor(true_labels)).float().sum().item() / len(true_labels)
print(f"Accuracy: {accuracy}")

In [ ]:
# 実際に予測されたラベルと正解ラベルを表示
for i in range(5):
    print(f"Text: {texts[i]}")
    print(f"model output: {categories[pred_labels[i]]}")
    print(f"True label: {categories[true_labels[i]]}")
    print()

### GPT-2(デコーダーモデル)を触ってみる

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
gpt_tokenizer.do_lower_case = True  

decoder_model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")
decoder_model.to(device)

### 各種パラメーターの説明
*   **max_length**<br>
生成されるテキストの最大長<br>
*   **top_k**<br>
出力候補の上位何％からランダムにピックアップするかを決めるパラメーター<br>
*   **temperature**<br>
高い確率の単語の可能性を増加させ、低確率の単語の可能性を減少させる。<br>
*   **do_sample**<br>
Top-Kサンプリングなどのランダムサンプリングを使用するかどうか。<br>
*   **num_beams**<br>
1よりも大きな値を指定することで、貪欲検索からビームサーチに切り替えられる。<br>
ビームサーチは、貪欲検索よりも多くの仮説を考慮する検索手法。この方法では、各ステップで複数の仮説を評価し、最終的にな確率が高くなるものを選びぶ。<br>
これにより、初期の確率が低いトークンで始まる高確率のシーケンスが貪欲検索によって無視されることがなくなる。<br>
*   **early_stopping**<br>
ビーム検索でEOSトークンに到達したときに生成が終了するように設定<br>
*   **no_repeat_ngram_size**<br>
指定されたngramサイズの繰り替えしへの制約を与える<br>

<br>
<br>
[参考]<br>
https://huggingface.co/docs/transformers/ja/generation_strategies<br>
https://zenn.dev/tyaahan/articles/a8d99900000002

In [ ]:
# 文章生成時のパラメータ
# TODO: パラメーターの値を変えたり追加したりして、モデルの出力がどのように変化するかをみてみましょう。各種パラメーターがどのような働きなのかを理解しながら変えてみるのが良いかもしれません。
params = {
    "max_length":100,
    "top_k" : 50,
    "temperature": 0.7,
    "do_sample": True,
    # "num_beams":5,
    # "early_stopping":True,
    # "no_repeat_ngram_size":2,
}


In [ ]:
# NOTE: start_textを色々変えてみると...?????
start_text = "昔々あるところに、"
with torch.no_grad():
    input_ids = gpt_tokenizer.encode(start_text, add_special_tokens=False, return_tensors="pt")
    output_ids = decoder_model.generate(input_ids.to(device), pad_token_id=gpt_tokenizer.pad_token_id, **params)

print(gpt_tokenizer.decode(output_ids.tolist()[0]))